## About the project
The [UBC EWS SkySpark platform](https://skyspark.energy.ubc.ca), managed by Energy and Water Services (EWS) of the University of British Columbi (UBC), collects data on weather and UBC buildings every 15 minutes. UBC Urban Data Lab (UDL) mirrored the SkySpark database into an InfluxDB instance to increase the accessibility and usability of the data.

UDL provides public users READ access to the InfluxDB with the credentials

`host='206.12.92.81',port=8086, username='public', password='public', database='SKYSPARK'`

You can access this InfluxDB database from [InfluxDB command line interface](https://docs.influxdata.com/influxdb/v1.7/tools/shell/) or [open-source InfluxDB API client libraries](https://docs.influxdata.com/influxdb/v1.7/tools/api_client_libraries/). This notebook demonstrates  querying the InfluxDB database using the `influxdb` Python library. Feel free to [contact UDL](https://data.sustain.ubc.ca/) if you have any questions.

In [1]:
import pandas as pd
import influxdb
from datetime import timezone, datetime
import pytz
import matplotlib.pyplot as plt
%matplotlib inline
import certifi

Querying the entire measurement `UBC_EWS` should be avoided.

```python
cli.query(query='select * from UBC_EWS')
#To return a large chunck of data can take very long or break the connection.
```

In [2]:
#DataFrameClient can write pandas DataFrames into influx, or read data into a pandas DataFrame
client = influxdb.DataFrameClient(host='206.12.92.81',port=8086, 
                                  username='public', password='public',database='SKYSPARK')
for d in client.get_list_measurements():
    print(d['name']) #d is a dictionary; 'name' is the key; measurement name is the value
#Use bind_params in a query
query = 'select * from UBC_EWS where siteRef=$siteRef and time > now() - 24h'
where_params = {'siteRef': 'Pharmacy'}
result = client.query(query = query, bind_params = where_params, 
                      chunked=True, chunk_size=10000)  #Use chunked responses from InfluxDB
                    # Try restart Python kernel and change chunk_size if it fails
df=result['UBC_EWS']
print("Time zone in InfluxDB:",df.index.tz)
my_timezone = pytz.timezone('Canada/Pacific')
df.index=df.index.tz_convert(my_timezone)
print("Converted to",my_timezone,"in dataframe")
pd.options.display.max_rows = 6
df

UBC_EWS
Time zone in InfluxDB: UTC
Converted to Canada/Pacific in dataframe


,equipRef,groupRef,navName,siteRef,typeRef,unit,value
2020-02-09 14:45:01-08:00,Rm 2342 VAV-2S032,Pharmacy Floor 2,Discharge Air Delta Pressure,Pharmacy,VAV_2S032_DP,Pa,-0.531624
2020-02-09 14:45:01-08:00,Rm 2342 VAV-2S032,Pharmacy Floor 2,Discharge Air Flow Sp,Pharmacy,VAV_2S032_FLW_SP,L/s,118.000000
2020-02-09 14:45:02-08:00,AHU-05,Pharmacy Air Systems,Entering Hot Water Temp,Pharmacy,AHU5_HWRT,°C,21.381865
...,...,...,...,...,...,...,...
2020-02-10 14:40:00-08:00,Cooling Plant CH-2,Pharmacy Hydronic Systems,CHLR2_COP_TL,Pharmacy,CHLR2_COP_TL,_,1.802154
2020-02-10 14:40:00-08:00,Cooling Plant CH-3,Pharmacy Hydronic Systems,CHLR3_COP_TL,Pharmacy,CHLR3_COP_TL,_,0.000000
2020-02-10 14:40:00-08:00,Cooling Plant CH-4,Pharmacy Hydronic Systems,CHLR4_COP_TL,Pharmacy,CHLR4_COP_TL,_,0.000000


In [5]:
Pharmacy = df
Pharmacy.index.name = 'time'
Pharmacy.reset_index(inplace=True)
print("Dataframe memory usage in bytes:",f"{Pharmacy.memory_usage().values.sum():,d}")
#Make a hierarchical DataFrame
#['siteRef','groupRef','equipRef','navName','typeRef'] (order can change) should uniquely identify a data stream.
Pharmacy.set_index(keys=['siteRef','groupRef','navName','typeRef','equipRef','unit'], inplace=True) 
Pharmacy.sort_index(inplace=True) #sort the index
pd.options.display.max_rows = 50
Pharmacy

Dataframe memory usage in bytes: 36,301,952


time  \
siteRef  groupRef             navName          typeRef          equipRef unit                             
Pharmacy Pharmacy Air Systems AHU10_13_HTG_BPV AHU10_13_HTG_BPV AHU-10   %    2020-02-09 17:10:51-08:00   
                                                                         %    2020-02-10 06:34:22-08:00   
                                                                         %    2020-02-10 07:24:55-08:00   
                                                                         %    2020-02-10 07:49:22-08:00   
                                                                         %    2020-02-10 07:57:06-08:00   
...                                                                                                 ...   
         Pharmacy Utilities   Water Flow       WM9_BLDG_FLOW    WM-9     _    2020-02-10 13:30:00-08:00   
                                                                         _    2020-02-10 13:45:00-08:00   
                                                                         _    2020-02-10 14:00:00-08:00   
                                                                         _    2020-02-10 14:15:00-08:00   
                                                                         _    2020-02-10 14:30:00-08:00   

                                                                                   value  
siteRef  groupRef             navName          typeRef          equipRef unit             
Pharmacy Pharmacy Air Systems AHU10_13_HTG_BPV AHU10_13_HTG_BPV AHU-10   %     74.652328  
                                                                         %     69.528160  
                                                                         %     64.453903  
                                                                         %     69.581451  
                                                                         %     74.651794  
...                                                                                  ...  
         Pharmacy Utilities   Water Flow       WM9_BLDG_FLOW    WM-9     _      0.000000  
                                                                         _      0.000000  
                                                                         _      0.000000  
                                                                         _      0.000000  
                                                                         _      0.000000  

[567216 rows x 2 columns]

In [7]:
#count values in each end-use group
pd.options.display.max_rows = 25
Pharmacy.groupby(by=['groupRef']).size().to_frame('count')

,count
groupRef,
Pharmacy Air Systems,134345
Pharmacy Floor 0,35101
Pharmacy Floor 1,23068
Pharmacy Floor 2,47556
Pharmacy Floor 3,79724
Pharmacy Floor 4,39510
Pharmacy Floor 5,39240
Pharmacy Floor 6,35210
Pharmacy Hydronic Systems,92840


In [49]:
#Pharmacy Floor 0
Floor0 = Pharmacy.iloc[Pharmacy.index.get_level_values('groupRef')=='Pharmacy Floor 0']
Floor0.groupby(by=['siteRef','groupRef','navName','typeRef','equipRef','unit']).size().to_frame('count').head(20)

count
siteRef  groupRef         navName                 typeRef                 equipRef           unit       
Pharmacy Pharmacy Floor 0 Alarm                   BE064_EXH_JAM_AL_BV     Rm B407a EAV-BE064 _        48
                                                  BE099_EXH_JAM_AL_BV     EAV-BE099          _        28
                                                  BS075_SUP_FLOW_AL_BV    Rm B411 EAV-BS075  _        44
                                                  BS096_SUP_FLOW_AL_BV    Rm B720 EAV-BS096  _        14
                                                  BS100_SUP_FLOW_AL_BV    Rm B411 EAV-BS100  _         8
                          BE001_SASH_OPEN_PERCENT BE001_SASH_OPEN_PERCENT EAV-BE001          %        24
                          BE002_SASH_OPEN_PERCENT BE002_SASH_OPEN_PERCENT EAV-BE002          %        24
                          BE008_SASH_OPEN_PERCENT BE008_SASH_OPEN_PERCENT EAV-BE008          %        24
                          BE030_EXH_OVRD_CMD_AV   BE030_EXH_OVRD_CMD_AV   Rm B608 EAV-BE030  %        95
                          BE064_EXH_OVRD_AV       BE064_EXH_OVRD_AV       Rm B407a EAV-BE064 _        48
                          BE064_EXH_OVRD_CMD_AV   BE064_EXH_OVRD_CMD_AV   Rm B407a EAV-BE064 %        95
                          BE068_SASH_OPEN_PERCENT BE068_SASH_OPEN_PERCENT Rm B409b EAV-BE068 %        24
                          BE068_VLV_FLW_FDBK      BE068_VLV_FLW_FDBK      Rm B409b EAV-BE068 L/s      95
                          BE091_SASH_OPEN_PERCENT BE091_SASH_OPEN_PERCENT Rm B503 EAV-BE091  %        24
                          BE093_EXH_OVRD_CMD_AV   BE093_EXH_OVRD_CMD_AV   EAV-BE093          %        95
                          BE107_EXH_OVRD_AV       BE107_EXH_OVRD_AV       Rm B306 EAV-BS092  _        95
                          BE116_EXH_OVRD_AV       BE116_EXH_OVRD_AV       Rm B025 EAV-BS087  _        95
                          BS001_PRIMARY_ACH       BS001_PRIMARY_ACH       Rm B407c EAV-BS001 _        95
                          BS001_ROOM_DIFF_AFLW    BS001_ROOM_DIFF_AFLW    Rm B407c EAV-BS001 L/s      95
                          BS002_PRIMARY_ACH       BS002_PRIMARY_ACH       Rm B407b EAV-BS002 _        95